In [1]:
using Plots
using Jacobi
using Test
using LinearAlgebra

In [2]:
function Eval(dim, x)

    result = tanh(x[1] + 0.1);
    if dim > 1
        result += atan(x[2] + 0.2);
    end
    if dim >2
        result += exp(-(x[3] + 0.3)*(x[3] + 0.3));
    end
    return result
end

Eval (generic function with 1 method)

In [3]:
function FEBasisEval(P, Q, nodes, qref1d)
"""
 FEBASISEVAL populates B1d and D1d of order P at quadrature points Q.
  input: 
        P: degree of Lagrange polynomial or shape functions
        Q: Number of quadrature points
 output:
        B1d: see above for description
        D1d: see above for description
"""
    B1d = zeros(1,P*Q);
    D1d = zeros(1,P*Q);

    for i = 1:Q
        c1 = 1.0;
        c3 = nodes[1] - qref1d[i];
        B1d[(i-1)*P+1] = 1.0;
        for j = 2:P
            c2 = 1.0;
            c4 = c3;
            c3 = nodes[j] - qref1d[i];
                for k = 1:j-1
                    dx = nodes[j] - nodes[k];
                    c2 = c2*dx;
                    if k == j-1
                        D1d[(i-1)*P + j] = c1*(B1d[(i-1)*P + k] - c4*D1d[(i-1)*P + k]) / c2;
                        B1d[(i-1)*P + j] = - c1*c4*B1d[(i-1)*P + k] / c2;
                    end
                    D1d[(i-1)*P + k] = (c3*D1d[(i-1)*P + k] - B1d[(i-1)*P + k]) / dx;
                    B1d[(i-1)*P + k] = c3*B1d[(i-1)*P + k] / dx;
                end
            c1 = c2;
        end
    end
    return B1d, D1d

end

function FEcreateBasis(P,Q, Qmode)
    """
 FECREATEBASIS creates and evaluates basis/shape function and their derivative 
 at quadrature points (Q) for the polynomial order P. 
  input: 
       P: polynomial degree + 1
       Q: Number of quadrature points
   Qmode: GAUSS or GLL quadrature points
 output:
     B1d: basis/shape function evaluated at quadrature points
     D1d: derivative of basis/shape function evaluated at quadrature points
       W: quadrature weights
  qref1d: quadrature points computed by GAUSS or LGL
    """
# get Legendre-Gauss-Lobatto nodes (or quadrature points)
      nodes = zglj(P, 0.0, 0.0); 
      if Qmode=="GAUSS"
         # populate a 1D array with GAUSS quadrature points and weights
        # 1D Gauss
        qref1d = zgj(Q, 0.0, 0.0)
        W = wgj(qref1d, 0.0, 0.0)
         # populate a 1D array for B1d and D1d described above in outputs
         B1d, D1d = FEBasisEval(P, Q, nodes, qref1d);
      elseif Qmode=="GLL"
         # populate a 1D array with GLL quadrature points and weights
         qref1d = zglj(Q, 0.0, 0.0)
         W = wglj(qref1d, 0.0, 0.0)  
         # populate a 1D array for B1d and D1d described above in outputs  
         B1d, D1d = FEBasisEval(P, Q, nodes, qref1d);
      else
         error("Qmode error! Choose GAUSS or GLL Quadrature points!");
      end
    return B1d, D1d, W, qref1d

end

FEcreateBasis (generic function with 1 method)

# Set dim, P, Q

In [4]:
dim = 3
P = 2
Q = 3
P_dim = P^dim
Q_dim = Q^dim

27

In [5]:
X_dim = 2^dim
X = zeros(X_dim*dim)
for d=1:dim
    for i=1:X_dim
        X[(d-1)*X_dim + i] = mod(i-1,2^(dim - (d-1))) ÷ 2^(dim - (d-1)-1) != 0 ? 1 : -1;
    end
end
X;

In [6]:
B1d, D1d, W, qref1d = FEcreateBasis(2, P, "GLL")
B_l = reshape(B1d,2,P)'
D_l = reshape(D1d,2,P)';

# dim-D basis, P = 2 (in 1D)
if dim == 1
    B_x = B_l
elseif dim == 2
    B_x = kron(B_l,B_l)
else
    B_x = kron(B_l,B_l, B_l)
end
# since num_comp = dim in the test we need another kron 
# CeedBasisCreateTensorH1Lagrange(ceed, dim, dim, 2, Q, CEED_GAUSS_LOBATTO,&basis_x_lobatto);
B_x = kron(I(dim), B_x);

In [7]:
# CeedBasisApply(basis_x_lobatto, 1, CEED_NOTRANSPOSE, CEED_EVAL_INTERP, X, X_q);
X_q = B_x*X;

In [12]:
xx = zeros(dim)
U = zeros(P_dim)
for i = 1:P_dim
    for d = 1:dim
        xx[d] = X_q[(d-1)*P_dim + i]
    end
    U[i] = Eval(dim, xx)
end
U;

# Check the tensor product for grad

In [13]:
B1d, D1d, W, qref1d = FEcreateBasis(P, Q, "GAUSS")
B = reshape(B1d,P,Q)'
D = reshape(D1d,P,Q)'
# dim-D basis , num_comp = 1
# CeedBasisCreateTensorH1Lagrange(ceed, dim, 1, P, Q, CEED_GAUSS,&basis_u_gauss);
if dim == 1
    G_u = D
    G_u = kron([1], G_u)
elseif dim == 2
    G_ux = kron(D,B)
    G_ux = kron([1, 0], G_ux)
    G_uy = kron(B,D)
    G_uy = kron([0, 1], G_uy)
    
    G_u = G_ux + G_uy
else
    G_ux = kron(D,B,B)  # Jed's paper
    #G_ux = kron(B,B,D) # libCEED
    G_ux = kron([1, 0, 0], G_ux)
    
    G_uy = kron(B,D,B)
    G_uy = kron([0, 1, 0], G_uy)
    
    G_uz = kron(B,B,D)  # Jed's paper
    #G_uz = kron(D,B,B) # libCEED
    G_uz = kron([0, 0, 1], G_uz)
    
    G_u = G_ux + G_uy + G_uz
end
# CeedBasisApply(basis_u_gauss, 1, CEED_NOTRANSPOSE, CEED_EVAL_GRAD, U, U_q);
U_q  = G_u * U

81-element Vector{Float64}:
 -0.2140534350957134
 -0.2140534350957134
 -0.2140534350957134
 -0.21405343509571337
 -0.21405343509571337
 -0.21405343509571337
 -0.21405343509571345
 -0.21405343509571345
 -0.21405343509571345
 -0.21405343509571342
 -0.21405343509571342
 -0.21405343509571342
 -0.2140534350957134
  ⋮
  0.758398445979827
  0.7583984459798272
  0.7583984459798271
  0.7583984459798272
  0.7583984459798271
  0.7583984459798271
  0.7583984459798271
  0.7583984459798272
  0.7583984459798271
  0.758398445979827
  0.7583984459798272
  0.7583984459798271

In [16]:
# CeedBasisApply(basis_u_gauss, 1, CEED_TRANSPOSE, CEED_EVAL_GRAD, ones, grad_T_ones);
grad_T_ones = G_u' * ones(Q_dim*dim)

8-element Vector{Float64}:
 -10.125
  -3.374999999999999
  -3.375
   3.3750000000000004
  -3.375
   3.375
   3.375
  10.124999999999996

In [17]:
sum_1 = 0;
for i=1:P_dim
    sum_1 += grad_T_ones[i]*U[i]
end

sum_2 = 0;
for i=1:Q_dim*dim
    sum_2 += U_q[i]
end

In [20]:
tol = 1E-5;

In [19]:
@test abs(sum_1- sum_2) < tol

Test Passed